## Lo primero que tengo que hacer es cargar el Dataframe del Notebook 1:

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

# Creo una carpeta para mi práctica en el directorio raíz
!mkdir -p "/content/drive/My Drive/Práctica NLP Najli YE"

Mounted at /content/drive


In [2]:
import pandas as pd

full_df = pd.read_csv('/content/drive/My Drive/Práctica NLP Najli YE/luxury_beauty_reviews.csv', sep=',', decimal='.')

Compruebo que la importación del csv haya ido bien:

In [3]:
full_df.head()

,overall,reviewText,sentiment_label
0,1.0,Mixed with water. I use the real rusk shampoo ...,0
1,1.0,I have used plumpers before and I can tell you...,0
2,1.0,The brush is splayed out so i can't really use...,0
3,1.0,"This soap is awful, it arrived leaking in its ...",0
4,1.0,does not work,0


In [4]:
full_df['reviewText'].head(10)

0    Mixed with water. I use the real rusk shampoo ...
1    I have used plumpers before and I can tell you...
2    The brush is splayed out so i can't really use...
3    This soap is awful, it arrived leaking in its ...
4                                        does not work
5    I get that that its heavy grease, but this see...
6    I have used this every night for a week with n...
7    I love CK one, but the deodorant is horrible, ...
8    I was so excited to have received this mascara...
9    I had a sample of the Oribe body wash. I absol...
Name: reviewText, dtype: object

Como es habitual, lo primero es instalar librerías:

In [5]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=4cf6a0a236b1b047dd3d4b16afbda98e7db9ee63825b368e738cc194d0c9306c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [6]:
import numpy as np
import pandas as pd
import unicodedata # Preprocesado
import re # Preprocesado
from num2words import num2words

In [7]:
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

Lo primero será tokenizar y podemos unificar todo a minúsculas

In [10]:
reviews = full_df['reviewText'].str.lower().str.split()

In [11]:
words = reviews.apply(pd.Series).stack().reset_index(drop = True)

In [12]:
from collections import Counter
vocabulario_ = Counter(words)
len(vocabulario_)

59394

Me genero un `lemmatizer` por si quiero usarlo en el preprocesado:

In [13]:
lemmatizer = WordNetLemmatizer()

Empiezo normalizando, eliminando signos de puntuación y otros símbolos, que ya vimos que aparecían en las reseñas:

In [14]:
def sentence_normalization(sentence):
    sentence = unicodedata.normalize('NFKD', sentence).lower().encode('ascii', errors='ignore').decode('utf-8')
    w = [re.sub(r'\W+', '', word) for word in sentence.split()]
    sentence = ' '.join([word for word in w if word.isalpha()]).strip()
    return sentence

Ahora las stopwords, las escojo de la librería de NLTK:

In [15]:
stopwrds_list = stopwords.words('english') # Nota: Stopwords todas en minúsculas

In [16]:
print(stopwrds_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
def remove_stopwords(sentence, sw_list):
    # Me aseguro de que las palabras pasen en minúsculas el chequeo de si son stopwords
    sentence = ' '.join([word for word in sentence.lower().split() if word not in sw_list])
    return sentence

Programo una única función de procesado que llama a su vez a las dos anteriores, `sentence_normalization` y `remove_stopwords`

In [18]:
def process_reviews(reviews, sw_list):
    processed_sentences = []

    for sent in reviews:
        if not sent != sent:  # check si sent no es NaN
            # Por las características de mi función de normalizacion, elimino 1º stopwords
            sent = remove_stopwords(sent, sw_list)

            # Ahora ya puedo normalizar
            sent = sentence_normalization(sent)

            # Añado palabra a la lista de procesadas
            processed_sentences.append(sent)
        else:
            processed_sentences.append('None')
    return processed_sentences

Aplico preprocesamiento de todas las reseñas y guardo en un nuevo df:

In [19]:
processed_reviews = process_reviews(full_df['reviewText'], stopwrds_list)

Contrastamos una review de ejemplo, antes y después del procesamiento:

In [20]:
print('Review original: {}'.format(full_df['reviewText'].values[0]))
print('Review procesada: {}'.format(processed_reviews[0]))

Review original: Mixed with water. I use the real rusk shampoo every week. This IS NOT THE SAME!!!!
Review procesada: mixed water use real rusk shampoo every week same


Otros ejemplos:

In [21]:
print('Review original: {}'.format(full_df['reviewText'].values[100]))
print('Review procesada: {}'.format(processed_reviews[100]))

Review original: Takes FOREVER to dry my thick curly hair. The diffuser (you have to buy separately) falls off all the time.  Invest your money in a different blow dryer because with this one, if you have thick curly hair, you'look be drying for about an hour.
Review procesada: takes forever dry thick curly hair diffuser you buy separately falls time invest money different blow dryer one thick curly hair youlook drying hour


In [22]:
print('Review original: {}'.format(full_df['reviewText'].values[20]))
print('Review procesada: {}'.format(processed_reviews[20]))

Review original: Doesn't work.
Review procesada: work


In [23]:
print('Review original: {}'.format(full_df['reviewText'].values[50]))
print('Review procesada: {}'.format(processed_reviews[50]))

Review original: I'm sure this would be great fresh but the tube I received was already past expiration and don't want to put it anywhere near my eyes. :-/
Review procesada: im sure would great fresh tube received already past expiration want put anywhere near eyes


Modifico el dataframe añadiendo la columna con la review procesada:

In [24]:
full_df.loc[:, 'processedReview'] = processed_reviews

Elimino los posibles valores nulos que haya:

In [25]:
full_df['processedReview'] = full_df['processedReview'].replace('', np.nan)
full_df = full_df.dropna(subset=['processedReview'])



---



Para poder seguir avanzando, guardo el df con las reviews procesadas en Drive:

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os

# Creo una carpeta para mi práctica en el directorio raíz
!mkdir -p "/content/drive/My Drive/Práctica NLP Najli YE"

In [28]:
# Guardo el df en un csv en la carpeta previamente creada
full_df.to_csv('/content/drive/My Drive/Práctica NLP Najli YE/lux_beauty_processed_reviews.csv', index=False)



---

